In [201]:
import matplotlib
%matplotlib qt5
from matplotlib.patches import Circle
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import numpy as np
import matplotlib.pylab as plt
import time

In [202]:
dx = dy = 0.005
dt = 0.1
D = 1800.00 * 1e-6
k = 14170.00 
R = 62.36*1e6 # Ideal gas constant, in mm3.mmHg.K-1.mol-1 
T = 309.25    # Blood temperature, 36.1*C in Kelvin
c0 = 68.50/(R*T) # Arterial oxygen concentration, in mol.mm-3
w = 1.00 *1e-3
U = 2400.00 *1e-6
x = np.arange(0,1,dx)
y = np.arange(0,0.5,dy)
X,Y = np.meshgrid(x,y)

In [203]:
nx = x.size
ny = y.size
coeffs = [dx/dy*D, dy/dx*D, 0, dy/dx*D, dx/dy*D]
coeffs[2] = -sum(coeffs)# - k*dx*dy
offsets = [-nx, -1, 0, 1, nx]
print(f"{nx=} {ny=}")

nx=200 ny=100


# Build reaction diffusion matrix and test 

In [204]:
M = sp.diags(coeffs, [-nx, -1, 0, 1, nx], shape=(nx*ny, nx*ny), format='lil')
for j in range(y.size):
    for i in range(x.size):
        cell = j*nx + i
        if i==0:
            M[cell,cell] += coeffs[1]
            if cell-1 >= 0:
                M[cell, cell-1] = 0.0
        if i==nx-1:
            M[cell, cell] += coeffs[3]
            if cell+1 < M.shape[1]:
                M[cell, cell+1] = 0.0
        if j==0:
            M[cell, cell] += coeffs[0]
        if j==ny-1:
            M[cell, cell] += coeffs[4]


In [205]:
# u = np.zeros(X.size)
deletecells = sp.eye(M.shape[0]).tolil()
# for j, yj in enumerate(y):
#     for i, xi in enumerate(x):
#         if (xi-0.5)**2 + (yj-0.5)**2 < 0.2**2:
#             cell = i + j*nx
#             deletecells[cell, cell] = 0.0
#             u[cell]= 1.0
u = np.zeros_like(X)
i0, i1 = int(7*ny/16), int(9*ny/16)
j0, j1 = int(4*nx/16), int(12*nx/16)
u[i0:i1, j0:j1] = 1.0
u = u.reshape((-1,))
for i in np.nonzero(u):
    deletecells[i,i] = 0

# print(f"{u.mean()=}, {u.min()=}, {u.max()=} {u.sum()=}")
# fig, ax = plt.subplots()
# c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
# fig.colorbar(c, ax=ax)
# plt.show()

In [206]:
# M = deletecells.dot(M)
# A = -dt*M + sp.eye(M.shape[0])
# A = A.tocsr()
# for k in range(10):
#     u = spl.spsolve(A, u)
#     print(f"{u.mean()=}, {u.min()=}, {u.max()=}")

In [207]:
# fig, ax = plt.subplots()
# c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
# fig.colorbar(c, ax=ax)
# circ = Circle((0.5,0.5), 0.2, fill=False, linestyle='--', linewidth=3)
# ax.add_patch(circ)
# plt.show()

# Build convection matrix

In [208]:
# Corners of the vessel
r = 0.05 # Radius
i0, i1 = int((y.max()/2-r/2)/dx), int((y.max()/2+r/2)/dx) # int(7*ny/16), int(9*ny/16)
j0, j1 = int(4*nx/16), int(12*nx/16)

In [209]:
nPts = j1-j0
nVol = nx*ny
C = sp.diags([1/dx,-1/dx], [0,1], shape=(nPts, nPts+nVol), format='lil')
C[nPts-1,nPts] = 0
C[0,1] = 0 # Will be a fixed inlet concentration
C = C.tocsr() 
# Add empty rows at the bottom of C to make it the right size
indptr = C.indptr.tolist()
for i in range(nVol):
    indptr.append(indptr[-1])
C = sp.csr_matrix((C.data, C.indices, indptr), shape=(nPts+nVol, nPts+nVol))

# Build matrix equating vascular voxel concentrations to nodal concentration

## Matrix linking vacsular voxels with vascular node

In [210]:
C4 = sp.lil_matrix((nVol, nPts))
I4 = np.zeros(nVol)
uVascularNodeToVoxel = np.zeros(nVol)
for node,j in enumerate(range(j0, j1)):
    for i in range(i0, i1):
        cell = i*nx+j
        C4[cell,node] = 1
        I4[cell] = 1
        uVascularNodeToVoxel[cell] = node+1
I4 = sp.diags([I4], [0]).tocsr()

# Check the loss
cv = np.ones(nPts) # Vascular nodal oxygen
print(np.any(I4.dot(u)-C4.dot(cv)!=0.0)) # Return False if each voxel's oxygen is equal to its corresponding node's oxygen 

# fig, ax = plt.subplots()
# c = ax.pcolormesh(uVascularNodeToVoxel.reshape(X.shape), cmap='RdBu', edgecolor='k')
# plt.colorbar(c, ax=ax, label='Linked to node')

False


## Build mass exchange between vessels and tissue

In [211]:
NodesToEndothelialCells = sp.lil_matrix((nPts, nPts+nVol))
for node, j in enumerate(range(j0, j1)):
    for i in [i0-1,i1]:
        cell = i*nx + j
        NodesToEndothelialCells[node, node] += -1
        NodesToEndothelialCells[node, nPts+cell] = 1
for i in range(i0, i1):
    for node, j in zip([0, nPts-1],[j0-1,j1]):
        cell = i*nx + j
        NodesToEndothelialCells[node, nPts+cell] = 1
        NodesToEndothelialCells[node, node] += -1        

In [212]:
Arr = NodesToEndothelialCells.toarray()
labels = np.zeros(nVol)
for j in range(j0, j1):
    for i in range(i0, i1):
        labels[i*nx+j] = 1

for node in range(nPts):
    labels[np.nonzero(Arr[node, nPts:])] = 2 

fig, ax = plt.subplots()
c = ax.pcolormesh(labels.reshape(X.shape), cmap='RdBu', edgecolor='k')
plt.colorbar(c, ax=ax, label='0 for tissue, 1 for vessel, 2 for endothelial cells')

In [213]:
V = x.max()*y.max() #dx*dy
Gamma = sp.diags([np.ones(nPts)*U*(j1-j0)*dy*np.pi*V/w], [0], shape=(nPts, nPts))
MassExchange = -NodesToEndothelialCells.T.dot(Gamma).dot(NodesToEndothelialCells)

# fig, ax = plt.subplots()
# c = ax.pcolormesh(abs(MassExchange.toarray()), cmap='RdBu')
# fig.colorbar(c, ax=ax)
# plt.show()

### Check if endothelial cells are connected to the right vascular nodes

In [214]:
Arr = NodesToEndothelialCells.toarray()
uCheckConnectionEndoToNode = uVascularNodeToVoxel
for node in range(nPts):
    uCheckConnectionEndoToNode[np.nonzero(Arr[node, nPts:])] = node+1

# fig, ax = plt.subplots()
# c = ax.pcolormesh(uCheckConnectionEndoToNode.reshape(X.shape), cmap='RdBu', edgecolor='k')
# plt.colorbar(c, ax=ax)
# plt.show()

### Make the consumption rate matrix and diffusion matrix (cells with no consumption/no diffusion need to be excluded)

In [215]:
deleteVascularCells = np.ones(nVol)
R = np.ones(nVol)*k*dx*dy
for i,label in enumerate(labels):
    if label==1:
        deleteVascularCells[i] = 0
    if label==2:
        R[i] /= k
deleteVascularCells = sp.diags([deleteVascularCells], [0])

R = sp.diags([R], [0])
R = deleteVascularCells.dot(R)
M = deleteVascularCells.dot(M)

In [216]:
print(nPts, nVol)
print(f"{MassExchange.shape=} {C.shape=} {M.shape=} {R.shape=} {C4.shape=} {I4.shape=}")
print(MassExchange.nnz, labels[labels>0].size)

100 20000
MassExchange.shape=(20100, 20100) C.shape=(20100, 20100) M.shape=(20000, 20000) R.shape=(20000, 20000) C4.shape=(20000, 100) I4.shape=(20000, 20000)
1220 1220


# Combine and reshape sub-matrices before combining them

In [217]:
# Equations associating vascular cells and nodes
M4 = sp.vstack([sp.csr_matrix((nPts, nPts+nVol)), sp.hstack([-C4, I4])])
# Reaction diffusion equations
Md = sp.vstack([sp.csr_matrix((nPts, nPts+nVol)), sp.hstack([sp.csr_matrix((nVol, nPts)), M-R])])
print(f"{MassExchange.shape=} {C.shape=} {Md.shape=} {M4.shape=}")

start_time = time.time()
K = C.tocsr() - MassExchange.tocsr() + Md.tocsr() + M4.tocsr()
print("--- %s seconds for matrix addition ---" % (time.time() - start_time))
# fig = plt.figure()
# plt.spy(K)

MassExchange.shape=(20100, 20100) C.shape=(20100, 20100) Md.shape=(20100, 20100) M4.shape=(20100, 20100)
--- 0.0008206367492675781 seconds for matrix addition ---


# Build the RHS vector

In [218]:
b = np.zeros(nPts+nVol)
# I = np.ones(nPts+nVol)
# I[:nPts] = 0
# K = sp.diags([I],[0], shape=K.shape).dot(K) 
# for i in range(nPts):
#     K[i,i] = 1
#     b[i] = c0

# 
b[0] = c0 # Inlet blood O2 concentration
# 
I = np.ones(nPts+nVol)
I[0] = 0
K = sp.diags([I],[0]).dot(K)
K[0,0] = 1.0

/usr/lib/python3/dist-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


# Solve the system

In [219]:
u = spl.spsolve(K, b)
ub, ut = u[:nPts], u[nPts:]

In [220]:
fig, ax = plt.subplots()
c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu')#, vmin=u.min(), vmax=u.max())
fig.colorbar(c, ax=ax)
plt.show()

In [221]:
# fig, ax = plt.subplots()
# c = ax.pcolormesh(K[:nPts, :].toarray(), cmap='RdBu')#, edgecolor='k')
# plt.colorbar(c, ax=ax)
# ax.set_aspect(nVol/nPts)
print('Tissue:', ut.mean(), ut.min(), ut.max(), ut.sum())
print('Blood: ', ub.mean(), ub.min(), ub.max(), ub.sum())

Tissue: 3.920679640436283e-12 -1.0497679865004125e-12 4.139599090340997e-09 7.841359280872566e-08
Blood:  3.5519771676307684e-11 -3.756958409517706e-14 3.552014718304491e-09 3.5519771676307686e-09


In [222]:
fig = plt.figure()
plt.plot(ub)